!pip list

In [1]:
from gpt4all import GPT4All

In [2]:
model = GPT4All('/app/ggml-gpt4all-j-v1.3-groovy.bin',)
model.model.set_thread_count(8)

Found model file at  /app/ggml-gpt4all-j-v1.3-groovy.bin
gptj_model_load: loading model from '/app/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [3]:
help(model.model._response_callback)

Help on function _response_callback in module gpt4all.pyllmodel:

_response_callback(token_id, response)
    # Empty response callback method that just prints response to be collected



In [14]:
%%time
model.generate(prompt.format(overview=data.overview[0]))

The Bachelor of Nursing Honours program is a specialized year after completing the Bachelor of Nursing degree, which focuses on research and professional development. It is designed to provide students with the skills and knowledge necessary for senior nursing or midwifery roles, as well as to pursue further education or research opportunities. The program offers a range of topics related to nursing practice and patient safety, including clinical risk management, chronic disease care and health delivery. Students will gain a deep understanding of the principles and ethics involved in health care research, as well as develop skills such as data management and research methods. The program also includes a range of practical skills, such
CPU times: user 10min 47s, sys: 4.37 s, total: 10min 51s
Wall time: 1min 24s


'The Bachelor of Nursing Honours program is a specialized year after completing the Bachelor of Nursing degree, which focuses on research and professional development. It is designed to provide students with the skills and knowledge necessary for senior nursing or midwifery roles, as well as to pursue further education or research opportunities. The program offers a range of topics related to nursing practice and patient safety, including clinical risk management, chronic disease care and health delivery. Students will gain a deep understanding of the principles and ethics involved in health care research, as well as develop skills such as data management and research methods. The program also includes a range of practical skills, such'

In [12]:
from pandas import read_csv, concat


def _callback(token_id: int, response: str) -> bool:
    global counter
    if response.decode('utf-8') == '.':
        # every time a . is detected is a new sentence
        counter += 1
    
    # if the total number of sentences are less than 2
    if counter < 2:
        
        # print them / process them - this will become calls to the /animateSay endpoint
        print(response.decode('utf-8'), end='')
        
        # continue generation process
        return True
    
    else:
        # reset sentence counter
        counter = 0
        print('.')
        
        # stop further generation
        return False


counter = 0
data = read_csv('course-data-small.csv')[['course_name', 'overview']]
model.model._response_callback = _callback

In [16]:
%%time
model.generate(
    prompt=prompt.format(overview=data.overview[0]), 
    streaming=True, temp=0.3,
)

The Bachelor of Nursing Honours is a specialized year after completing an undergraduate degree in nursing. It is designed to provide students with the knowledge and research skills necessary for senior nursing roles or advanced professional training.

CPU times: user 10min 27s, sys: 3.02 s, total: 10min 30s
Wall time: 1min 19s


'The Bachelor of Nursing Honours is a specialized year after completing an undergraduate degree in nursing. It is designed to provide students with the knowledge and research skills necessary for senior nursing roles or advanced professional training.\n'

In [1]:
from langchain.chains           import RetrievalQA
from langchain.document_loaders import DataFrameLoader
from langchain.llms             import GPT4All
from langchain.embeddings       import HuggingFaceEmbeddings
from langchain.text_splitter    import CharacterTextSplitter
from langchain.vectorstores     import Chroma
from langchain.prompts          import PromptTemplate

from langchain.chains.summarize import load_summarize_chain
from langchain.callbacks.base   import BaseCallbackHandler


import pandas as pd
from pandas import read_csv, concat

In [4]:
def capitalize(text):
    return " ".join([word.capitalize() for word in text.split()])


class MainCallback(BaseCallbackHandler):

    def __init__(self, max_sentences=5, *args, **kwargs):
        super().__init__(*args, **kwargs) 
        self.max_sentences = max_sentences
        self.num_sentences = 0
        self.raise_error = True

    def on_llm_new_token(self, token: str, **kwargs):
        if token == '.':
            self.num_sentences += 1
            if self.num_sentences >= self.max_sentences:
                print('Max senteces reached: ', self.num_sentences)
            raise Exception('Dumb Exception.')
        else:
            try:
                token.encode('utf-8')
            except UnicodeDecodeError as e:
                # take ascii equivalent of unicode char
                token = ord(token)
                    
            # print('TOKEN: ', type(token), token, capitalize(token))
    
    def on_llm_end(self, response, *, run_id, parent_run_id=None, **kwargs):
        print('The number of sentences is: ', self.num_sentences)
        


callback = MainCallback(max_sentences=5)

NameError: name 'BaseCallbackHandler' is not defined

In [2]:
data = read_csv('course-data-small.csv')[['course_name', 'overview']]
data['content'] = data['course_name'] + ' ' + data.overview
data['content'] = data.content.str.replace('’', '').str.replace('^', ' ')
loader = DataFrameLoader(data[['content']], page_content_column='content')
document = loader.load_and_split()

In [3]:
# embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
# db = Chroma.from_documents(document, embeddings,)
model_path = '/app/ggml-gpt4all-j-v1.3-groovy.bin'
llm = GPT4All(model=model_path, verbose=False, n_threads=8, seed=42, temp=0.3, streaming=False, use_mlock=True)

Found model file at  /app/ggml-gpt4all-j-v1.3-groovy.bin
gptj_model_load: loading model from '/app/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [12]:
llm.client.model._response_callback(0, 'Test'.encode())
llm.client.model._response_callback(0, 'Test'.encode())
llm.client.model._response_callback(0, 'Test'.encode())
llm.client.model._response_callback(0, 'Test'.encode())
llm.client.model._response_callback(0, 'Test'.encode())
llm.client.model._response_callback(0, 'Test'.encode())

TestTestTestTestTestTest

True

In [5]:
llm.use_mlock

False

In [4]:
dir(llm)

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '_abc_impl',
 '_acall',
 '_agenerate',
 '_calculate_keys',
 '_call',
 '_c

document[0]

%%time
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
temp = chain.run([document[0]])

In [14]:
from langchain import PromptTemplate

template = """
    Summarize this overview in as few words as possible.
    {overview}
    In summary, 
"""
prompt = PromptTemplate(
    input_variables=["overview"], template=template
)

In [15]:
prompt.format(overview=data.overview[0])

'\n    Summarize this overview in as few words as possible.\n    Focusing on research, the honours year builds on the foundations established in your undergraduate degree. You will study a particular topic of professional interest in greater depth and create pathways to specialisations and further research or study. The Bachelor of Nursing (Honours) is a specialised year of study taken after the completion of a Bachelor of Nursing, entry to practice Master degree or combined degrees. For entry into senior nursing and midwifery roles there’s now an increasing industry expectation for graduates with high-level research and evaluation skills. Honours offers you a competitive edge in the job market and is designed to provide you with the knowledge and research skills to undertake a postgraduate research degree, advanced professional training or to pursue diverse employment opportunities. The School of Nursing and Midwifery offers research in areas influencing quality and patient safety in 

In [7]:
data = read_csv('course-data-small.csv')[['course_name', 'overview']].reset_index(drop=True)

temp = {
    'course_name': [], 
    'o_summarized': [], 
    'overview': []
}

result_df = pd.DataFrame(temp)
result_df.to_csv('gpt-summarized-info.csv', index=False)
counter = 0
for idx, doc in data.iterrows():
    temp['course_name'].append(doc.course_name)
    temp['overview'].append(doc.overview)
    temp['o_summarized'].append(
        llm.predict(
            prompt.format(overview=doc.overview),
        )
    )
    # save dataframe every 5 rows
    counter += 1

    # Save the data to the DataFrame every 5 rows
    if counter % 5 == 0:
        pd.DataFrame(temp).to_csv('gpt-summarized-info.csv', mode='a', index=False, header=False)
        temp = {'course_name': [], 'o_summarized': [], 'overview': []}
        print(f'\rCompleted IDX: {idx} \t| Counter: {counter}', end='', flush=True)
        
    break

temp

The Bachelor's Degree is an undergraduate program that provides students with foundational knowledge about nursing practice through hands-on clinical experiences in hospitals/health facilities where they learn from experienced nurses who are experts at their field. The focus on the Honours Year, which follows after completion of a bachelor degree and before entry into professional programs such as Master��s or Doctoral degrees is to provide students with specialized knowledge about nursing practice through hands-on clinical experiences in hospitals/health facilities where they learn from experienced nurses who are experts at their field. The Bachelor's Degree program focuses on providing foundational knowledge, skills for the Honours Year and preparing them well enough so that when it comes time to enter professional programs like Master��s or Doctoral degrees, students will have a strong foundation in nursing practice with specialized expertise they can apply towards further education

Exception: Dumb Exception.